In [2]:
import matplotlib
matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180

import sys
sys.path.append("/home/cat/code/widefield/") # go to parent dir

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import glob2

from sklearn import linear_model
from scipy import stats
import numpy as np
import matplotlib.patches as mpatches

# visualize results module
from visualize import visualize_svm as Visualize

# 
#data_dir = '/media/cat/4TBSSD/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [ ]:
#######################################################################
#######################################################################
#######################################################################
def compute_significance(data,
                     significance):

    #print ("self.data: ", data.shape)

    mean = data.mean(1)
    #
    sig = []
    for k in range(data.shape[0]):
        #res = stats.ks_2samp(self.data[k],
        #                     control)
        #res = stats.ttest_ind(first, second, axis=0, equal_var=True)

        #
        res = scipy.stats.ttest_1samp(data[k], 0.5)

        sig.append(res[1])


    sig_save = np.array(sig).copy()
    #print ("Self sig save: ", sig_save.shape)

    # multiple hypothesis test
    temp = np.array(sig)
    #print ("data into multi-hypothesis tes:", temp.shape)
    temp2 = multipletests(temp, alpha=significance, method='fdr_bh')
    sig = temp2[1]

    #
    sig=np.array(sig)[None]

    #
    thresh = significance
    idx = np.where(sig>thresh)
    sig[idx] = np.nan

    #
    idx = np.where(mean<0.5)
    sig[:,idx] = np.nan
    #print ("SIG: ", sig.shape)

    # find earliest 
    earliest_continuous = 0
    for k in range(sig.shape[1]-1,0,-1):
        if sig[0][k]<=significance:
            earliest_continuous = k
        else:
            break

    earliest_continuous = -(sig.shape[1]-earliest_continuous)/30.

    return sig, earliest_continuous
    
    
class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """

    def __init__(self, *args, **kwargs):
        if not "fit_intercept" in kwargs:
            kwargs['fit_intercept'] = False
        super(LinearRegression, self)\
                .__init__(*args, **kwargs)

    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)

        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([
            np.sqrt(np.diagonal(sse[i] * np.linalg.inv(np.dot(X.T, X))))
                                                    for i in range(sse.shape[0])
                    ])

        self.t = self.coef_ / se
        self.p = 2 * (1 - stats.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1]))
        return self
    
 

In [ ]:
###############################################################
####### GENERATE LONGITUDINAL SVM CURVES (VIRIDIS PLOTS) ######
###############################################################

# Fig 4b from paper longitudinal viridis plots

# 6 panels with SVM curves in vidiris; ALREADY GEENERATED
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

# 
fig, ax = plt.subplots()
import scipy
from statsmodels.stats.multitest import multipletests

# 
smooth = True
smooth_window = 30
significance = 0.05
auc = []
early = []
from tqdm import tqdm, trange
for k, name in enumerate(names):
    fnames = np.loadtxt('/media/cat/4TBSSD/yuki/'+name+'/concatenated_svm.txt',dtype='str') #[::-1]

    ax=plt.subplot(2,3,k+1)
    colors = plt.cm.viridis(np.linspace(0,1,len(fnames)))

    auc.append([])
    early.append([])
    for ctr,fname in tqdm(enumerate(fnames)):

        data = np.load(fname, allow_pickle=True)
        acc = data['accuracy']

        if smooth:            
            data = []
            for p in range(acc.shape[1]):
                box = np.ones(smooth_window)/smooth_window
                trace_smooth = np.convolve(acc[:,p], 
                                           box, 
                                           mode='valid')

                data.append(trace_smooth)
            data = np.array(data)
            acc = np.array(data).copy().T

        # 
        mean = acc.mean(1)

        t = np.arange(mean.shape[0])/30-29
        ax.plot(t,mean,
                linewidth=2,
                color=colors[ctr])
    
        # append area under cruve up to t = -10sec
        #print (mean.shape)
        temp = mean[-300:]  # last 10sec prior to movement
        auc[k].append(temp.sum())
        
        sig, earliest = compute_significance(data.T,
                                             significance)
        early[k].append(earliest)
        
    plt.ylim(0.4,1.0)
    plt.xlim(-15,0)
    plt.plot([-30,0],[0.5,0.5],'--',c='black')

if True:
    plt.savefig('/home/cat/edt_trends.svg',dpi=1200)
    plt.close()

In [ ]:
#########################################
########### EDT FITS ####################
#########################################

# Fig 4c 

colors = ['black','blue','red','green','magenta','pink','cyan']
ids= ["M1","M2","M3","M4","M5","M6",]
fig=plt.figure(figsize=(6,6))
for k in range(len(early)):
#for k in [3]:
    ax=plt.subplot(3,2,k+1)
    temp = np.array(early[k])
    idx =np.where(temp<-20)[0]
    temp[idx]=temp[idx-1]
    t = np.arange(temp.shape[0])/temp.shape[0]
    
    #
    lr = LinearRegression()
    lr.fit(t.reshape(-1, 1), temp.reshape(-1, 1))
    #print (k, temp)
    #print (k,"tsta: ", lr.t, "pval: ", lr.p)


    
    temp2 = np.poly1d(np.polyfit(t, temp, 1))(t)
    plt.plot(t, temp2,
             linewidth=6,
             #label=ids[k]+ " ***** ",
             c='black')

    # 
    corr = scipy.stats.pearsonr(t,temp)
    print ("corrL: ", corr)
    
    plt.scatter(t,
                temp, 
                #label=names[k]+ " "+str(round(corr[0],2))+
                #                        " ("+str("pval: {0:.1}".format(corr[1]))+")",
                s=100,
                #linewidth=4,
                c='grey',
                alpha=.8,
               label = str(round(corr[0],2)))
    #t = np.arange(auc1.shape[0])/(auc1.shape[0]-1)

    plt.xticks([])
    plt.yticks([])
    legend = plt.legend(handlelength=0, fontsize=12)
    
    #plt.legend(fontsize=12)
    plt.ylim(-12,0)

#
    plt.xlim(0,1)

if True:
    plt.savefig('/home/cat/edt_fits.svg', dpi=600)
    plt.close()
else:
    plt.show()


In [ ]:
######################################################
######################################################
######################################################

fig=plt.figure(figsize=(5,5))
for k in range(6):
    ax=plt.subplot(3,2,k+1)
    y = cons[k]
    x = np.arange(y.shape[0])/y.shape[0]
    
    t = stats.pearsonr(x, y)
    print (t)
    plt.scatter(x,y, c='black', alpha=.5,
                label = str(np.round(t[0],2)))
    
    
    
    # 
    coef = np.polyfit(x,y,1)

    # 
    poly1d_fn = np.poly1d(coef) 
    yy = poly1d_fn(x)
    plt.plot(x, yy, '--',
             c='black',
            linewidth=3) #'--k'=black dashed line, 'yo' = yellow circle marker
    plt.xticks([])
    plt.yticks([])

    plt.legend(fontsize=20)
    plt.ylim(-15,0)  
    plt.xlim(0,x[-1])
    
if True:
    plt.show()
else:
    plt.savefig('/home/cat/longitudinal_concatead_edts.svg')
    plt.close()
    

In [ ]:
#########################################
########### AUC AND EARLY PLOTS #########
#########################################

# FIg 3J 3I

colors = ['black','blue','red','green','magenta','pink','cyan']
ids= ["M1","M2","M3","M4","M5","M6",]
fig2=plt.figure()
for k in range(len(early)):
    temp = np.array(early[k])
    idx =np.where(temp<-20)[0]
    temp[idx]=temp[idx-1]
    t = np.arange(temp.shape[0])
    
    #
    temp2 = np.poly1d(np.polyfit(t, temp, 1))(t)
    plt.plot(t, temp2,
             linewidth=6,
             label=ids[k]+ " ***** ",

             c=colors[k])

    # 
    corr = scipy.stats.pearsonr(t,temp2)
    print ("corr: ", corr)
    
    
    plt.scatter(t,
                temp, 
                #label=names[k]+ " "+str(round(corr[0],2))+
                #                        " ("+str("pval: {0:.1}".format(corr[1]))+")",
                s=100,
                linewidth=4,
                c=colors[k],
                alpha=.3)
    #t = np.arange(auc1.shape[0])/(auc1.shape[0]-1)


#
plt.xlim(0,80)
plt.ylim(-12,0)
plt.legend(fontsize=20)
plt.show()
